## Presentation

A notebook to process BCDI data using cdiutils package and PyNX phase retrieval

In [ ]:
import os
import hdf5plugin # if data are too large
from xrayutilities import en2lam

from cdiutils.process.pipeline import BcdiPipeline
from cdiutils.process.parameters import get_parameters_from_notebook_variables

## Processing parameters

In [ ]:
metadata = {
  "beamline_setup": "",  # Required
  "scan": None,  # Required
  "sample_name": "",  # Required for ID01BLISS, P10 and SIXS2022 setups
  "experiment_file_path": "",  # Required for ID01SPEC, ID01BLISS setups
  "experiment_data_dir_path": "",  # Required for P10 and SIXS2022 setups
  "detector_data_path": "",  # Required for ID01SPEC setup
  "edf_file_template": "",  # Required for ID01SPEC setup
  "detector_name": "",  # Required
  "flat_field": None,
  "alien_mask": None,
  "dump_dir": "",
  "reconstruction_file": "mode.h5"
}

# Required. You choose: either you specify it in the metadata, or you
# want the dump_dir to be dependent on the 'sample_name' and 'scan' or
# other things. 
metadata["dump_dir"] = (
    os.getcwd() + f'/results/{metadata["sample_name"]}/S{metadata["scan"]}/'
)

preprocessing_output_shape = []  # Required, 2 or 3 values. If 2, will take the whole RC 
energy = None  # Required, in eV
hkl = []  # Required

# Required, must be a list of "com", "max" or tuple of int that corresponds
# to the position you want to crop the data at. It can be 2D. And you can
# mix it with binning_along_axis0. 
det_reference_voxel_method = []  # Ex: [(70, 200, 200), "com", "com"]
binning_along_axis0 = None  # Whether or not you want to bin in the RC direction
light_loading = False  # Load only the roi defined by the det_reference_voxel and preprocessing_output_shape

# Required
det_calib_parameters = {
  "cch1": 0,  # direct beam vertical position 
  "cch2": 0,  # horizontal
  "pwidth1": 5.5e-05,  # detector pixel size in m
  "pwidth2": 5.5e-05,  # detector pixel size in m
  "distance": 0,  # sample to detector distance in m
  "tiltazimuth": 0,
  "tilt": 0,
  "detrot": 0,
  "outerangle_offset": 0.0
}

voxel_size = None
apodize = "blackman"
flip = False
isosurface = None

# Display parameters
usetex = True  # might not work if running the notebook directly on nice or slurm
show = True
verbose = True
debug = False

# Support and surface determination parameters, optional
method_det_support = "Isosurface" # "Isosurface" or "Amplitude_variation"
order_of_derivative = "Gradient" # "Gradient" or "Laplacian"
raw_process = True # False if you want to fill the holes in the surface, True otherwise

# Facet determination parameters, optional
nb_facets = 32 # Expected number of facets of the particle
top_facet_reference_index = [1,1,1]
authorized_index = ['max', 1]  # ['max', n], ['absolute', a, b, c, ...] or ['families', [h1, k1, l1],[h2, k2, l2], ...]
remove_edges = True

# Optional
nb_nghbs_min = 0
index_to_display = None # None for the top facet or [[h1,k1,l1],[h2,k2,l2],...]
display_f_e_c = 'facet' # 'facet', 'edge', 'corner', or 'all'
size = 10 #Size of the voxels in the visualization
support_path = None

# PyNX parameters

# You can leave as it is
data = (
  metadata["dump_dir"]
  + "/pynx_phasing/"
  + f"S{metadata['scan']}_pynx_input_data.npz"
)

mask = (
  metadata["dump_dir"]
  + "/pynx_phasing/"
  + f"S{metadata['scan']}_pynx_input_mask.npz"
)

data2cxi = True

# support_size is the radius or half-size for the initial support. 
# Either one value (will be attributed to all dimensions), or one value 
# for each dimension. To be used in combination with support (which must 
# be different to "auto"). If support is "auto", leave support_size to
# None.
support = "auto"
support_size = None  

support_threshold = "0.30, 0.40"
support_threshold_method = "rms"
support_only_shrink = False
support_update_period = 20
support_smooth_width_begin = 2
support_smooth_width_end = 1
support_post_expand = "1,-2,1"
psf = "pseudo-voigt,0.5,0.1,10"

nb_raar = 500
nb_hio = 300
nb_er = 200
nb_ml = 0
nb_run = 15
nb_run_keep = 10

zero_mask = False
crop_output = 0
positivity = False
beta = 0.9
detwin = False

rebin = "1,1,1"
detector_distance = det_calib_parameters["distance"]
pixel_size_detector = det_calib_parameters["pwidth1"]
wavelength = float(en2lam(energy) * 1e-10)  # wavelength in m

verbose = 100
output_format = "cxi"
live_plot = False
save_plot = True
mpi = "run"


# Load the parameters and parse them into the BcdiPipeline class instance 
parameters = get_parameters_from_notebook_variables(dir(), globals())
bcdi_pipeline = BcdiPipeline(parameters=parameters)

### Preprocess the data: crop and center the data and compute orthogonalization parameters

In [ ]:
bcdi_pipeline.preprocess()

### Run PyNX phase retrieval using ESRF's p9 GPUs or another machine

In [ ]:
# Here you can update PyNX parameters before running the phasing

# bcdi_pipeline.params["pynx"].update(
#     {
#         "rebin": "1,1,1",
#         "support_update"_period": 20,
#         "support_threshold": "0.15, 0.40"
#     }
# )

user = os.environ["USER"]
key_file_path = os.environ["HOME"] + "/.ssh/id_rsa"
number_of_nodes = 2

print(
    f"Will use the user name '{user}' "
    f"and the private key file path:\n'{key_file_path}'"
)

bcdi_pipeline.phase_retrieval(
    machine="slurm-nice-devel",
    # machine="lid01pwr9",
    user=user,
    number_of_nodes=number_of_nodes,
    key_file_path=key_file_path,
    remove_last_results=True
)

### Analyse the phasing results

In [ ]:
bcdi_pipeline.analyse_phasing_results(
    sorting_criterion="sharpness",
    # plot_phasing_results=True,  # Default is False
    # plot_amplitude=True  # Default is False
)

### Select the best reconstructions decompose them into one mode

In [ ]:
# Chose the number of best candidates you want to keep.
number_of_best_candidates: int = 5

# You can either select the best candidates by specifying the reconstruction
# number
bcdi_pipeline.select_best_candidates(
    # best_runs=[10]
    nb_of_best_sorted_runs=number_of_best_candidates
)

bcdi_pipeline.mode_decomposition()

### Finally postprocess the data: orthogonalize, compute phase, dispacement, strain, dspacing...

In [ ]:
# Here you can update a few post-processing parameters instead of going
# back up to the top of the notebook.

# This is whether you want to play with support/surface determination
# By default the tradional method "Isosurface" is used, so you do not need
# to update the following parameters
# bcdi_pipeline.params["cdiutils"].update(
#     {
#         "method_det_support": "Isosurface",  # "Isosurface" or "Amplitude_variation"
#         "order_of_derivative": "Gradient",  # "Gradient" or "Laplacian"
#         "raw_process" : True,
#     }
# )

# However, you can play with these
bcdi_pipeline.params["cdiutils"].update(
    {
        "isosurface": 0.4,
        "voxel_size": 10,
        "flip": False,
        "apodize": "blackman",
        "handle_defects": False
    }
)

bcdi_pipeline.postprocess()

### Facet analysis

In [ ]:
# Here you can update a few post-processing parameters instead of going
# back up to the top of the notebook, ex:

bcdi_pipeline.params["cdiutils"].update(
    {
        "nb_facets": 26, # Expected number of facets of the particle
        "remove_edges": True,
        "nb_nghbs_min": 0,
        "authorized_index": ["max", 1],  # ['max',n],['absolute',a,b,c,...] or ['families',[h1,k1,l1],[h2,k2,l2],...]
        "index_to_display": [[1,1,1]],  # None for the top facet or [[h1,k1,l1],[h2,k2,l2],...]
        "display_f_e_c": "facet",  # 'facet', 'edge', 'corner', or 'all'
        "size": 10  # Size of the voxels in the visualization
    }
)

bcdi_pipeline.facet_analysis()